In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
from tfyolo3.dataloaders import common
from tfyolo3.helpers import draw
from tfyolo3 import YoloV3
from tfyolo3 import layers
from pathlib import Path
import numpy as np
import tensorflow as tf
import math
import numpy as np
from matplotlib import patches
import matplotlib.pyplot as plt

In [ ]:
classes = common.load_classes('../tests/data/classes.txt')
# classes.append('car interior')
classes


In [ ]:
target_shape = (512, 512, 3)
batch_size = 4
# grid_len = 8
is_training = True
max_boxes_per_image = 50
num_classes = len(classes)

In [ ]:
model = YoloV3(target_shape,
               max_boxes_per_image,
               # anchors=YoloV3.default_anchors,
               num_classes=num_classes,training=False)

In [ ]:
w_path = str(Path('../weights.117-9.932.h5').absolute())
model.load_weights(w_path)

In [ ]:
tf.keras.utils.plot_model(model.model, show_shapes=True)

In [ ]:
img = common.open_image('./test.jpg')
img_pad = common.pad_to_fixed_size(img, target_shape)
img_resized = common.resize(img, target_shape)
x = np.divide(img_pad, 255.)
x = np.expand_dims(x, 0)
x.shape

In [ ]:
boxes, scores, classes, sel = model.model.predict(x)

In [ ]:
boxes[:,:sel[0],:]

In [ ]:
scores

In [ ]:
classes

In [ ]:
sel

In [ ]:
class_names = Path('../model_data/coco_classes.txt')\
    .read_text().splitlines()
class_names_dict = dict(enumerate(class_names))

In [ ]:
ax = draw.show_img(img_resized, figsize=(16,10))
for i,b in enumerate(boxes[0,:sel[0]]):
    draw.rect(ax, b)
    draw.text(ax, b[:2], class_names_dict[classes[0, i]] + ' ' + str(round(scores[0,i],2)), sz=10)
    print(class_names_dict[classes[0, i]], scores[0,i])
    
plt.show()